# Carla 03

## Conda Setup

Please run the following from `Anaconda Powershell Prompt` (or any other terminal with `conda` installed).

From Lab PC, you can find it in the start menu or in the Desktop.

### 1. Environment creation

Create a new environment with the following command (you can skip this step if you already have a conda environment):

```bash
conda create -n carla-env python=3.7
```

### 2. Environment activation

Activate the environment with the following command:

```bash
conda activate carla-env
```

### 3. Package installation

Install the required packages with the following command (run it where the `requirements.txt` file is located):

```bash
pip install -r requirements.txt
```

### 4. VSCode setup

If you are using VSCode, you can select the conda environment by clicking on the bottom left corner:

`Select kernel > Python Environments > carla-env`

Let vscode install the required packages for you.

In [ ]:
import carla, time, pygame, math, random, cv2
import numpy as np

In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

world = client.get_world()
spectator = world.get_spectator()

## Some helpful functions used in this notebook

In [ ]:
def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), width=800, height=600):
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

## IDE suggestions

In VSCode, you can get suggestions for functions and classes by pressing `Ctrl + Space`.

In [ ]:
map = world.get_map()
map

Unfortunately, carla does not provide docs for the functions and classes, so we need to install it manually.

It can be done in VSCode with the following the instructions (resumed below):

1. Go to the repo https://github.com/aasewold/carla-python-stubs
2. Go to the releases section
3. Download `*.pyi` file (i.e `__init__.pyi` and `command.pyi`)  that match carla version (e.g 0.9.15)
4. Put the file in `./typings/carla` ([vscode reference](https://code.visualstudio.com/docs/python/settings-reference))

> **Note**: `./typings/carla` should be relative to the root of the vscode project

In [ ]:
map

## Sensors

Sensors are another type of actors, usually spawned attached to a vehicle, designed to retrieve data from the world. The type of data depends on the sensor, but it can range from RGB images, LiDAR scans or even collision information.

Sensors are divided into two main categories:

- **Regular sensors**: these sensors retrieve data at a fixed rate, usually every tick (e.g RGB cameras).
- **Trigger sensors**: these sensors only retrieve data when a certain condition is met (e.g collision sensors).

Sensors details can be found in the [CARLA documentation](https://carla.readthedocs.io/en/latest/ref_sensors/).

![Sensors](img/sensors.jpg)

### Exploring sensors

In [ ]:
sensors = world.get_blueprint_library().filter('sensor.*')

for sensor in sensors:
    print(sensor.id)

## Camera sensor

In [ ]:
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')

camera_bp.set_attribute('image_size_x', '1280')
camera_bp.set_attribute('image_size_y', '720')
camera_bp.set_attribute('fov', '120')
camera_bp.set_attribute('sensor_tick', '0')

spawn_point = carla.Transform()
camera = world.spawn_actor(camera_bp, spawn_point)

time.sleep(1)
move_spectator_to(camera.get_transform())

# count = 0
# def handle_image(image):
#     global count
#     image.save_to_disk(f'output/{count}.png')
#     count += 1

# camera.listen(lambda image: handle_image(image))

time.sleep(10)
camera.destroy()

Let's spawn a vehicle and a camera in a particular position and see what the camera sees.

In [ ]:
camera_position = carla.Transform(carla.Location(x=-34, y=31, z=11), carla.Rotation(pitch=-18, yaw=-170, roll=0))

camera = spawn_camera(transform=camera_position)
vehicle = spawn_vehicle()

camera.listen(lambda image: image.save_to_disk('output/camera.png'))

time.sleep(2)

camera.destroy()
vehicle.destroy()

### Attach a camera to a vehicle

In order to acquire images from the point of view of a vehicle, we can attach a camera to it and retrieve what it sees, based on the camera's position and orientation. When the camera is attached to a vehicle, it will move with the vehicle itself.

In [ ]:
vehicle = spawn_vehicle()
camera = spawn_camera(attach_to=vehicle)

camera.listen(lambda image: image.save_to_disk(f'output/{image.frame}.png'))
vehicle.set_autopilot(True)

time.sleep(10)

camera.destroy()
vehicle.destroy()

How many images have been captured by the camera so far?

### Live camera feed

We can also visualize the camera feed in real-time. This is useful for debugging and understanding what the camera sees.

We can use the `opencv` library to create a window and display the camera feed.

In [ ]:
vehicle = spawn_vehicle()
camera = spawn_camera(attach_to=vehicle)

video_output = np.zeros((600, 800, 4), dtype=np.uint8)
def camera_callback(image):
    global video_output
    video_output = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

camera.listen(lambda image: camera_callback(image))

vehicle.set_autopilot(True)

cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)

running = True

try:
    while running:
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        cv2.imshow('RGB Camera', video_output)
finally:
    cv2.destroyAllWindows()
    camera.destroy()
    vehicle.destroy()

## Exercise 1 - Multi-camera setup

This exercise involves creating a multi-camera setup in the CARLA simulator to simulate a car's mirror and driver point-of-view system using four different cameras. The objective is to capture video streams from different perspectives around a vehicle, mimicking how a driver would view their surroundings through the front windshield, side mirrors, and rearview mirror. You should position each cameras with specific transformations and resolutions, and display each video in individual OpenCV windows.

This is an example of output:

![Multi-camera setup](img/camera-position-exercise-output.png)

## Exercise 2 - Automatic light control

Implement a script that allow the vehicle to turn on its lights when it gets dark and turn them off when it gets bright, using the camera sensor.

To change the weather conditions, we can use the `PythonAPI\util\config.py` script.

```bash
python ./config.py --weather ClearNoon
python ./config.py --weather ClearNight
```

## Exercise 3 - Pacman effect using GPS

Implement the Pacman effect for a vehicle using a GPS sensor for tracking the vehicle's position. You can start from the implementation of the pacman effect using the vehicle's position below.

In [ ]:
vehicle = spawn_vehicle(world)

ROAD_X_MIN = -110
ROAD_X_MAX = 110

try:
    while True:
        transform = vehicle.get_transform()
        location = transform.location

        print(f"Location: {location.x}, {location.y}", end='\r')

        if location.x > ROAD_X_MAX:
            location.x = ROAD_X_MIN
            vehicle.set_transform(carla.Transform(location, transform.rotation))

        control = carla.VehicleControl(throttle=0.5)
        vehicle.apply_control(control)

        world.tick()
except KeyboardInterrupt:
    pass
finally:
    vehicle.destroy()

## Exercise 4 - Emergency braking system using LIDAR

LIDAR (Light Detection and Ranging) is a method for determining ranges by targeting an object or a surface with a laser and measuring the time for the reflected light to return to the receiver.

LIDAR sensors are used in autonomous vehicles to detect obstacles and create a 3D representation of the environment.

Implment a script that detect obstacles in front of the vehicle and apply the brakes if the distance is less than a threshold, using the LIDAR sensor.

Some notes on the simulation environment to test the emergency braking system:

- Spawn a vehicle (i.e ego vehicle)
- Spawn a vehicle (i.e target vehicle) an position it in front of the ego vehicle
- Attach a LIDAR sensor to the ego vehicle
- Create the logic to detect obstacles in front of the ego vehicle, based on the LIDAR sensor data. You can find the documentation for the LIDAR sensor [here](https://carla.readthedocs.io/en/latest/ref_sensors/#lidar-sensor).
- Apply the brakes if the distance is less than a threshold.